In [1]:
!pip install -U -q PyDrive
# !apt-get install libopenblas-dev 

# !conda create --name lightnet python=3  --yes
# !source activate lightnet 
# !pip install pathlib
# !pip install numpy
# !pip install plac
# !pip install requests
# !pip install msgpack-python
# !pip install lightnet

!apt update
!apt install -y cmake
!pip install dlib

Hit:1 http://security.ubuntu.com/ubuntu artful-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu artful InRelease
Hit:3 http://archive.ubuntu.com/ubuntu artful-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu artful-backports InRelease
Reading package lists... Done
Building dependency tree       
Reading state information... Done
11 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
cmake is already the newest version (3.9.1-1).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.


In [2]:
!pip install face_recognition
import face_recognition

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def getFile(file_name, target_id):

  # 1. Authenticate and create the PyDrive client.
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  #2. Get the file
  downloaded = drive.CreateFile({'id':target_id}) # replace the id with id of file you want to access
  downloaded.GetContentFile(file_name)  
  print 'done downloading '+ file_name

In [6]:
# TODO for test video
files_X = ['test_vid_1.avi']
files_Y = ['labels_1.csv']
files_X_key = ['1LwZBim4W_aAS52ADH_JNkMMwMlwvboKq']
files_Y_key = ['15YNRctoaFQCELUibW13sDgb-UDqwJZBm']

for i in range(0, len(files_X)):
  getFile(files_X[i], files_X_key[i])
  getFile(files_Y[i], files_Y_key[i])

done downloading test_vid_1.avi
done downloading labels_1.csv


In [7]:
model_file = ['my_model.h5', 'my_model-noisefree.h5']
model_file_key = ['1RyCOwl57SgIj9sZKhIQdhda0M4YAAGZV','1j44Fh24rCuVJKY5oChQcNoaavsIZU9RO']
getFile(model_file[1], model_file_key[1])

done downloading my_model-noisefree.h5


In [0]:
y_vals = {
    'SG':0,
    'FR':1,
    'AK':2,
    'SP':3,
    'SK':4,
    'SM':5,
    'jk':6
}

In [0]:
import numpy as np
import csv
y_list = list()
with open(files_Y[0]) as f:
  reader = csv.reader(f)
  for row in reader:
    s = ''
    for ab in row:
      s = s + ab
    y_list.append(y_vals[s])

y_list = np.array(y_list)

In [0]:
import cv2
import numpy as np
import PIL.Image
import matplotlib.pyplot as plt
from cStringIO import StringIO
import IPython.display
from tempfile import TemporaryFile
def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [11]:
!python -m lightnet download yolo

/usr/bin/python2: No module named lightnet


In [0]:
# import lightnet
# from lightnet import Image

# model = lightnet.load('yolo')


In [0]:
# def detectPerson(img):

#   boxes = model(img)
# #   print (boxes)
#   return boxes

In [0]:
def cropImages(boxes, img):
  imageList = list()
  for box in boxes:
    
    
      
    x1 = box[0]
    y1 = box[1]
    x2 = box[2]
    y2 = box[3]

#     x = max(int(center_x - width/2), 0)
#     y = max(int(center_y - height/2),0)
#     width = int(width)
#     height = int(height)
    # Crop!!
    crop_img = img[min(x1, x2):max(x2, x1), min(y1, y2):max(y1, y2)]
#       showarray(crop_img)
    try:
      crop_img = cv2.resize(crop_img, (150, 120))
    except cv2.error as e:
      print x1,y1,x2, y2
      print 'could not crop'
      break

    imageList.append(crop_img)


    
  return imageList

In [0]:
import face_recognition
def test(video_path, labels):
    vidcap = cv2.VideoCapture(video_path)
    correct = 0
    count = 0
    k = 0
    success = True
    # print vidcap
    while success:
        
        success, image = vidcap.read()
        count += 1
        if count%20 == 0:
          print correct, count, k
#         img = cv2.resize(image, image_shape)
        img = image
        cv2.imwrite('frame1.jpg', img)
#         im = lightnet.Image.from_bytes(open('frame1.jpg', 'rb').read())
        image2 = face_recognition.load_image_file("frame1.jpg")
        face_locations = face_recognition.face_locations(image2, model='cnn')
#         boxes = detectPerson(im)
        imList = cropImages(face_locations, img)
        if len(imList) == 0:
          if labels[count-1] == 6:
            correct = correct + 1
          continue
#         showarray(imList[0])
#         imList = np.array(imList, dtype=np.float32)
        imList = np.array(imList, dtype=np.float32)/255.0
#         print imList
        
        y = model_net.predict(imList)
        
        preds = np.argmax(y, axis=1)
#         print y
        for pred in preds:
          if pred == labels[count-1]:
            correct += 1
            break
            
            
    acc = float(correct)/count
    print 'acc : ', acc

In [15]:
from keras.models import load_model

model_net = load_model('my_model-noisefree.h5')

Using TensorFlow backend.


In [16]:
test('test_vid_1.avi', y_list)

10 20 0
19 40 0
29 60 0
40 80 0
49 100 0
58 120 0
73 140 0
84 160 0
94 180 0
101 200 0
112 220 0
116 240 0
122 260 0
129 280 0
137 300 0
147 320 0
154 340 0
161 360 0
169 380 0
177 400 0
188 420 0
196 440 0
205 460 0
212 480 0
225 500 0
240 520 0
251 540 0
262 560 0
271 580 0
280 600 0
286 620 0
293 640 0
302 660 0
314 680 0
323 700 0
329 720 0
339 740 0
352 760 0
359 780 0
369 800 0
377 820 0
384 840 0
393 860 0
403 880 0
412 900 0
421 920 0
430 940 0
443 960 0
450 980 0
459 1000 0
471 1020 0
478 1040 0
485 1060 0
495 1080 0
505 1100 0
513 1120 0
523 1140 0
532 1160 0
542 1180 0
551 1200 0
562 1220 0
571 1240 0
579 1260 0
589 1280 0
594 1300 0


KeyboardInterrupt: ignored

In [53]:
from google.colab import files
uploaded = files.upload()

KeyboardInterrupt: ignored

In [0]:
def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer = model.layers[-4]
    return layer